# Game Recommendation System

In [36]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [37]:
user_data = pd.read_csv('./datasets/users.csv')
game_data = pd.read_csv('./datasets/games.csv')
# recommendation_data = pd.read_csv('./datasets/recommendations.csv')
games_metadata = pd.read_json('./datasets/games_metadata.json', lines=True)


In [38]:
user_data.head()

,user_id,products,reviews
0,7360263,359,0
1,14020781,156,1
2,8762579,329,4
3,4820647,176,4
4,5167327,98,2


In [39]:
game_data.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [40]:
games_metadata.head()

,app_id,description,tags
0,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
1,22364,,[Action]
2,113020,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
3,226560,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
4,249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."


## Data Preprocessing

In [41]:
user_data.isnull().sum()

user_id     0
products    0
reviews     0
dtype: int64

In [42]:
game_data.isnull().sum()

app_id            0
title             0
date_release      0
win               0
mac               0
linux             0
rating            0
positive_ratio    0
user_reviews      0
price_final       0
price_original    0
discount          0
steam_deck        0
dtype: int64

In [43]:
games_metadata.isnull().sum()

app_id         0
description    0
tags           0
dtype: int64

In [44]:
# Merge datasets on 'app_id'
game_data = game_data.merge(games_metadata, on='app_id', how='left')


In [45]:
# If tags are lists, join them into a single string
game_data['tags'] = game_data['tags'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [46]:
# Combine tags into a single string for textual representation
game_data['combined_features'] = game_data['tags']


## Feature Representation

In [47]:
# Convert tags into a feature matrix using CountVectorizer
vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(game_data['combined_features'])

In [48]:
# Convert feature_matrix to a sparse CSR matrix for memory efficiency
feature_matrix_sparse = csr_matrix(feature_matrix)

## Representation Function

In [49]:
def recommend_games(game_title, num_recommendations=5):
    # Find the index of the game in the dataset
    game_index = game_data[game_data['title'] == game_title].index[0]

    # Compute cosine similarity scores only for the target game
    similarity_scores = cosine_similarity(feature_matrix_sparse[game_index], feature_matrix_sparse).flatten()

    # Sort games by similarity scores (descending)
    sorted_indices = np.argsort(-similarity_scores)

    # Fetch the top N similar games (excluding the input game itself)
    recommended_games = [
        game_data.iloc[i]['title']
        for i in sorted_indices[1:num_recommendations + 1]
    ]

    return recommended_games


## Example Usage

In [ ]:
if __name__ == "__main__":
    game_to_recommend = "Escape Dead Island"  # Replace with any game title from the dataset
    recommendations = recommend_games(game_to_recommend, num_recommendations=5)
    print(f"Games similar to '{game_to_recommend}':")
    for i, game in enumerate(recommendations, 1):
        print(f"{i}. {game}")

Games similar to 'Escape Dead Island':
1. Sniper Elite: Nazi Zombie Army
2. How To Survive: Third Person Standalone
3. Cold Fear™
4. Sniper Elite: Nazi Zombie Army 2
5. The Evil Within 2
